## 提示模版
根据用户输入的固定模版字符串的组合创建提示对象

In [ ]:
import os
import json
import dashscope
from dashscope import Generation
from langchain import PromptTemplate

# 读取配置文件
with open("config.json", "r") as f:
    config = json.load(f)
    api_key = config["api_key"]

# 设置DashScope API密钥
os.environ['DASHSCOPE_API_KEY'] = api_key
dashscope.api_key = api_key

def chat_with_qwen(prompt):
    response = Generation.call(
        model=config['qwen_model'],
        prompt=prompt
    )
    if response.status_code == 200:
        return response.output.text
    else:
        raise Exception(f"API调用失败: {response.code} - {response.message}")


template = """
    请推荐一本适合 {reader} 的书籍，主题围绕 {topic}，风格偏向 {style}，内容需包含 {keyword}。
    书籍应具备 [特点，如启发性、趣味性、深度等]，并适合在 [场景，如休闲阅读、学习研究等] 下阅读。
    请简要说明推荐理由，并附上书籍的基本信息（如书名、作者、出版年份）。
"""

prompt =  PromptTemplate(   
    template=template,
    input_variables=["reader", "topic", "style", "keyword"]
)

final_prompt = prompt.format(reader="程序员", topic="编程", style="技术", keyword="Python")

print("最终提示:", final_prompt)
response = chat_with_qwen(final_prompt)
print("回复:", response)

最终提示: 
    请推荐一本适合 程序员 的书籍，主题围绕 编程，风格偏向 技术，内容需包含 Python。
    书籍应具备 [特点，如启发性、趣味性、深度等]，并适合在 [场景，如休闲阅读、学习研究等] 下阅读。
    请简要说明推荐理由，并附上书籍的基本信息（如书名、作者、出版年份）。

回复: **推荐书籍：《Python编程：从入门到实践》**

### 推荐理由：
这本书非常适合程序员作为学习和研究的参考书。它兼具启发性和趣味性，同时内容深入浅出，能够帮助读者快速掌握 Python 编程的核心概念和技术应用。书中不仅涵盖了基础语法和常用库的使用，还通过实际项目案例（如构建 Web 应用程序、数据可视化等）让读者将理论知识应用于实践。无论是初学者还是有一定经验的开发者，都能从中受益。

此外，这本书的内容组织合理，既适合休闲阅读以激发兴趣，也适合系统学习以提升技能。对于希望深入了解 Python 并将其应用于实际问题解决的程序员来说，这是一本非常实用且值得推荐的书籍。

### 基本信息：
- **书名**：《Python编程：从入门到实践》  
- **作者**：Eric Matthes  
- **出版年份**：2016  

这本书适合在以下场景下阅读：
- 学习研究：帮助初学者快速上手 Python，并为进阶开发者提供丰富的实践案例。
- 实战演练：通过项目驱动的方式巩固所学知识，提高解决问题的能力。

如果你希望在轻松愉快的过程中学习一门强大的编程语言，《Python编程：从入门到实践》无疑是一个理想的选择！


# 管道提示词
管道提示词可以将多个提示组合在一起。当您想要重复使用部分提示时,这会很有用。这可以通过PipelinePrompt来完成。
PipelinePrompt由两个主要部分组成:
- 最终提示:返回的最终提示
- 管道提示:元组列表,由字符串名称和提示模板组成。每个提示模板将被格式化,然后作为具有相同名称的变量传递到未来的提示模板。

In [11]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

# 最终模版
final_template = """
{introduction}

{example}

{start_generation}
"""

# 最终提示
final_prompt = PromptTemplate.from_template(final_template)


# 介绍模版
introduction_template = """
你正在扮演{person}。
"""

# 介绍提示
introduction_prompt = PromptTemplate.from_template(introduction_template)

# 示例模版
example_template = """
下面是一个交互示例：
Q：{question}
A：{answer}
"""

# 示例提示
example_prompt = PromptTemplate.from_template(example_template)

# 开始生成模版
start_generation_template = """
现在正式开始回答问题！

Q：{input}
A：
"""

# 开始生成提示
start_generation_prompt = PromptTemplate.from_template(start_generation_template)

# 管道模版
pipeline_template = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start_generation", start_generation_prompt)
]

# 创建管道提示
pipeline_prompt = PipelinePromptTemplate(
    pipeline_prompts=pipeline_template,
    final_prompt=final_prompt,
    streaming = True
)

import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model= "qwen-turbo",
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("API_BASEURL"),
    streaming= True
)

# 创建链
chain = pipeline_prompt | llm

# 使用 stream 方法进行流式输出
for chunk in chain.stream({
    "person": "李白",
    "question": "写一首关于春天的诗",
    "answer": "春眠不觉晓，处处闻啼鸟。夜来风雨声，花落知多少。",
    "input": "写一首关于夏天的诗"
}):
    print(chunk.content, end="", flush=True)


夏风拂柳影，荷香满池塘。
日暮倚修竹，心闲岁月长。
蝉鸣惊远客，蜻蜓立浅滩。
悠然见南山，此景最难忘。